In [11]:
import pandas as pd
import numpy as np
import os
from itertools import combinations # create combinations of inputs# Seaborn visualization library
import time
from statsmodels.tools import add_constant # add a constant (bias) term to linear regression
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS # Ordinary Linear Regression
from statsmodels.tools import add_constant # add a constant (bias) term to linear regression

In [2]:
df = pd.read_csv('data/BRFSS_clean.csv')
df

,Unnamed: 0,SEQNO,MENTHLTH,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,DIAB,...,RETIR,UNAB_WORK,M,F,WHITE,BLACK,ASIAN,AM_IND,HISP,ANOTHER_RACE
0,0,b'2021000001',10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1,0,0,1,1,0,0,0,0,0
1,2,b'2021000003',0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,1,0,1,0,0,0,0
2,3,b'2021000004',10.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,1,1,0,0,0,0,0
3,5,b'2021000006',0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,0,1,0,1,0,0,0,0,0
4,9,b'2021000010',5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260632,438686,b'2021001379',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,0,0
260633,438687,b'2021001380',0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,0,1,0,0,0,0,0,1,0
260634,438688,b'2021001381',20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,1,0,0,0,0
260635,438691,b'2021001384',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,1,0,0,0,0


In [7]:
y = df['MENTHLTH']
x = df.iloc[:,3:]
x

,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,DIAB,HEALTHSCORE,PHYSHLTH,HIGHBP,...,RETIR,UNAB_WORK,M,F,WHITE,BLACK,ASIAN,AM_IND,HISP,ANOTHER_RACE
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,20.0,3.0,...,1,0,0,1,1,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1,0,0,1,0,1,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1,0,0,1,1,0,0,0,0,0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,3.0,...,1,0,1,0,1,0,0,0,0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,25.0,1.0,...,1,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,...,0,0,1,0,0,1,0,0,0,0
260633,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,...,1,0,1,0,0,0,0,0,1,0
260634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,1,0,1,0,0,0,0
260635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,1,0,0,1,0,0,0,0


In [3]:
from mpmath import mp
mp.dps = 50
class BMA2:
    
    def __init__(self, y, X, **kwargs):
        # Setup the basic variables.
        self.y = y
        self.X = X
        self.names = list(X.columns)
        self.nRows, self.nCols = np.shape(X)
        self.likelihoods = mp.zeros(self.nCols,1)
        self.likelihoods_all = {}
        self.coefficients_mp = mp.zeros(self.nCols,1)
        self.coefficients = np.zeros(self.nCols)
        self.probabilities = np.zeros(self.nCols)
        # Check the max model size. (Max number of predictor variables to use in a model.)
        # This can be used to reduce the runtime but not doing an exhaustive sampling.
        if 'MaxVars' in kwargs.keys():
            self.MaxVars = kwargs['MaxVars']
        else:
            self.MaxVars = self.nCols  
        # Prepare the priors if they are provided.
        # The priors are provided for the individual regressor variables.
        # The prior for a model is the product of the priors on the variables in the model.
        if 'Priors' in kwargs.keys():
            if np.size(kwargs['Priors']) == self.nCols:
                self.Priors = kwargs['Priors']
            else:
                print("WARNING: Provided priors error.  Using equal priors instead.")
                print("The priors should be a numpy array of length equal tot he number of regressor variables.")
                self.Priors = np.ones(self.nCols)  
        else:
            self.Priors = np.ones(self.nCols)  
        if 'Verbose' in kwargs.keys():
            self.Verbose = kwargs['Verbose'] 
        else:
            self.Verbose = False 
        if 'RegType' in kwargs.keys():
            self.RegType = kwargs['RegType'] 
        else:
            self.RegType = 'LS' 
        
    def fit(self):
        # Perform the Bayesian Model Averaging
        
        # Initialize the sum of the likelihoods for all the models to zero.  
        # This will be the 'normalization' denominator in Bayes Theorem.
        likelighood_sum = 0
        
        # To facilitate iterating through all possible models, we start by iterating thorugh
        # the number of elements in the model.  
        max_likelihood = 0
        for num_elements in range(1,self.MaxVars+1): 
            
            if self.Verbose == True:
                print("Computing BMA for models of size: ", num_elements)
            
            # Make a list of all index sets of models of this size.
            Models_next = list(combinations(list(range(self.nCols)), num_elements)) 
             
            # Occam's window - compute the candidate models to use for the next iteration
            # Models_previous: the set of models from the previous iteration that satisfy (likelihhod > max_likelihhod/20)
            # Models_next:     the set of candidate models for the next iteration
            # Models_current:  the set of models from Models_next that can be consturcted by adding one new variable
            #                    to a model from Models_previous
            if num_elements == 1:
                Models_current = Models_next
                Models_previous = []
            else:
                idx_keep = np.zeros(len(Models_next))
                for M_new,idx in zip(Models_next,range(len(Models_next))):
                    for M_good in Models_previous:
                        if(all(x in M_new for x in M_good)):
                            idx_keep[idx] = 1
                            break
                        else:
                            pass
                Models_current = np.asarray(Models_next)[np.where(idx_keep==1)].tolist()
                Models_previous = []
                        
            
            # Iterate through all possible models of the given size.
            for model_index_set in Models_current:
                
                # Compute the linear regression for this given model. 
                model_X = self.X.iloc[:,list(model_index_set)]
                if self.RegType == 'Logit':
                    model_regr = sm.Logit(self.y, model_X).fit(disp=0)
                else:
                    model_regr = OLS(self.y, model_X).fit()
                
                # Compute the likelihood (times the prior) for the model. 
                model_likelihood = mp.exp(-model_regr.bic/2)*np.prod(self.Priors[list(model_index_set)])
                    
                if (model_likelihood > max_likelihood/20):
                    if self.Verbose == True:
                        print("Model Variables:",model_index_set,"likelihood=",model_likelihood)
                    self.likelihoods_all[str(model_index_set)] = model_likelihood
                    
                    # Add this likelihood to the running tally of likelihoods.
                    likelighood_sum = mp.fadd(likelighood_sum, model_likelihood)

                    # Add this likelihood (times the priors) to the running tally
                    # of likelihoods for each variable in the model.
                    for idx, i in zip(model_index_set, range(num_elements)):
                        self.likelihoods[idx] = mp.fadd(self.likelihoods[idx], model_likelihood, prec=1000)
                        self.coefficients_mp[idx] = mp.fadd(self.coefficients_mp[idx], model_regr.params[i]*model_likelihood, prec=1000)
                    Models_previous.append(model_index_set) # add this model to the list of good models
                    max_likelihood = np.max([max_likelihood,model_likelihood]) # get the new max likelihood if it is this model
                else:
                    if self.Verbose == True:
                        print("Model Variables:",model_index_set,"rejected by Occam's window")
                    

        # Divide by the denominator in Bayes theorem to normalize the probabilities 
        # sum to one.
        self.likelighood_sum = likelighood_sum
        for idx in range(self.nCols):
            self.probabilities[idx] = mp.fdiv(self.likelihoods[idx],likelighood_sum, prec=1000)
            self.coefficients[idx] = mp.fdiv(self.coefficients_mp[idx],likelighood_sum, prec=1000)
        
        # Return the new BMA object as an output.
        return self
    
    def predict(self, data):
        data = np.asarray(data)
        if self.RegType == 'Logit':
            try:
                result = 1/(1+np.exp(-1*np.dot(self.coefficients,data)))
            except:
                result = 1/(1+np.exp(-1*np.dot(self.coefficients,data.T)))
        else:
            try:
                result = np.dot(self.coefficients,data)
            except:
                result = np.dot(self.coefficients,data.T)
        
        return result  
        
    def summary(self):
        # Return the BMA results as a data frame for easy viewing.
        df = pd.DataFrame([self.names, list(self.probabilities), list(self.coefficients)], 
             ["Variable Name", "Probability", "Avg. Coefficient"]).T
        return df

In [17]:
result=BMA2(y, add_constant(x), MaxVars=7)
result.fit()

In [18]:
result.summary()

,Variable Name,Probability,Avg. Coefficient
0,const,1.0,4.826037
1,DEAF,0.0,0.0
2,BLIND,0.0,0.0
3,DECIDE,1.0,7.385779
4,DIFFWALK,0.0,0.0
5,DIFFDRES,0.0,0.0
6,DIFFALON,0.0,0.0
7,DIAB,0.0,0.0
8,HEALTHSCORE,1.0,0.430915
9,PHYSHLTH,1.0,0.192135
